# Nuz Klassifir

### Download NLTK packages

In [125]:
import nltk
#nltk.download()

### Training Data Collection and Preprocessing

In [126]:
import os, re
direk_folders = ['culture', 'diverse', 'economy', 'internationalNews', 'localNews', 'politic', 'society', 'sport', 'technology' ]

In [135]:
train = []
max_samples = 100

for folder_name in direk_folders:
    direk = 'train/{}/'.format(folder_name)
    current_files_names = os.listdir(direk)
    
    i = 0
    for file_name in current_files_names:
        
        # stop at max number of samples to avoid bias for some classes
        i += 1
        if i == max_samples:
            break
            
        # read file
        file_reader = open(direk + file_name, "r", encoding="utf-8")
        file = file_reader.read()

        # lowercase
        file = file.lower()
        file = file.replace('\r', ' ').replace('\n', ' ')

        # extract title text
        title = re.search(r'<title>(.*?)</(.?)title>', file)
        cleanr = re.compile('<.*?>')
        clean_title = re.sub(cleanr, '', title.group(0))
    
        # extract abstract text
        abstr = re.search(r'<abstract>(.*?)</(.?)abstract>', file)
        clean_abstr = re.sub(cleanr, '', abstr.group(0))

        # extract text of the main content
        text = re.search(r'<text>(.*?)</(.?)text>', file)
        clean_text = re.sub(cleanr, '', text.group(0))
     
        clean_input = clean_title + clean_abstr + clean_text
        train.append([clean_input, folder_name])

### URL, special characters, and digits removal

In [136]:
for row in train:
    row[0] = re.sub(r'http\S+', '', row[0])
    # remove special characters and digits
    row[0] = re.sub("(\\d|\\W)+"," ", row[0])
print(train[:3])

[['مخرج غدوة حي الفيلم يجمع التشويق و المغامرة و الفرجة و الموسيقى استضاف برنامج سينما سينما الأربعاء جانفي في موعد استثنائي أبطال الفيلم التونسي الطويل غدوة حي demain dès l aube و مخرج العمل لطفي عاشور هذه الاستضافة تأتي في إطار انطلاق عرض الفيلم على مستوى الجهات و البداية كانت بقاعة السينما الوحيدة في سوسة و يقول المخرج التونسي لطفي عاشور إن أحداث الفيلم تتمثل في تحقيق قضائي لحادثة وقعت ليلة جانفي و تتواصل أطوارها بعد سنوات من هذا التاريخ بأسلوب فيه الكثير من التشويق و المغامرة و الفرجة و توظيف الموسيقى بشكل مدروس و دقيق وهو ما أثار إعجاب المشاهدين حسب قوله من بين الحضور أي ضا أشرف بن يوسف الذي عب ر عن إعجابه بخوض تجربة التمثيل لأول مر ة رفقة نخبة من الممثلين الذين شاركهم دور البطولة في الفيلم مضيفا أن ه اشتغل على تطوير أداءه قبل خوض هذه التجربة من جهتها بطلة غدوة حي الممثل ة أنيسة داود قالت إن أحداث الفيلم تروي قصة شخصيات أجبرتهم الأحداث التي عاشوها ليلة جانفي على أن ي صبحوا أصدقاء رغم التناقضات مشيرة إلى أن المشاهد سيجد نفسه أمام أداء ممي ز لبعض الوجوه الشابة و سيعيش لحظات طريفة من

### Stemming

In [137]:
from nltk.stem.porter import *
from nltk import word_tokenize
stemmer = PorterStemmer()
#nltk.download('punkt')

In [138]:
for row in train:
    #tokenize
    splitted = word_tokenize(row[0])

    for i in range(0, len(splitted)):
        splitted[i] = stemmer.stem(splitted[i])
    
    row[0] = splitted
print(train[:3])

[[['مخرج', 'غدوة', 'حي', 'الفيلم', 'يجمع', 'التشويق', 'و', 'المغامرة', 'و', 'الفرجة', 'و', 'الموسيقى', 'استضاف', 'برنامج', 'سينما', 'سينما', 'الأربعاء', 'جانفي', 'في', 'موعد', 'استثنائي', 'أبطال', 'الفيلم', 'التونسي', 'الطويل', 'غدوة', 'حي', 'demain', 'dè', 'l', 'aub', 'و', 'مخرج', 'العمل', 'لطفي', 'عاشور', 'هذه', 'الاستضافة', 'تأتي', 'في', 'إطار', 'انطلاق', 'عرض', 'الفيلم', 'على', 'مستوى', 'الجهات', 'و', 'البداية', 'كانت', 'بقاعة', 'السينما', 'الوحيدة', 'في', 'سوسة', 'و', 'يقول', 'المخرج', 'التونسي', 'لطفي', 'عاشور', 'إن', 'أحداث', 'الفيلم', 'تتمثل', 'في', 'تحقيق', 'قضائي', 'لحادثة', 'وقعت', 'ليلة', 'جانفي', 'و', 'تتواصل', 'أطوارها', 'بعد', 'سنوات', 'من', 'هذا', 'التاريخ', 'بأسلوب', 'فيه', 'الكثير', 'من', 'التشويق', 'و', 'المغامرة', 'و', 'الفرجة', 'و', 'توظيف', 'الموسيقى', 'بشكل', 'مدروس', 'و', 'دقيق', 'وهو', 'ما', 'أثار', 'إعجاب', 'المشاهدين', 'حسب', 'قوله', 'من', 'بين', 'الحضور', 'أي', 'ضا', 'أشرف', 'بن', 'يوسف', 'الذي', 'عب', 'ر', 'عن', 'إعجابه', 'بخوض', 'تجربة', 'التمثيل', 'لأول',

### Stop-word removal

In [139]:
#nltk.download('stopwords')
from nltk.corpus import stopwords

In [143]:
stop_words = set(stopwords.words('arabic')) 
for row in train:
    x = [w for w in row[0] if not w in stop_words]
    row[0] = " ".join(x)

### Model

In [144]:
from sklearn.feature_extraction.text import TfidfVectorizer

train_sent = [line[0] for line in train]
train_labels = [line[1] for line in train]

tdif_vec = TfidfVectorizer(norm = None)
tfidf = tdif_vec.fit_transform(train_sent)

### Feed the training documents and their labels to a NB classifier

In [145]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB(alpha=1)
clf.fit(tfidf, train_labels)

MultinomialNB(alpha=1, class_prior=None, fit_prior=True)

### Test Data Collection and Preprocessing

In [146]:
import json
json_file = open('test\posts.json', encoding='utf8')
json_reader = json.load(json_file)

user_interest = []

test = []
for user in json_reader.values():
    for article in user:
        #use the title and description as input
        dirty_input = article[0] + article[1]
        
        # lowercase and remove line breaks
        clean_input = dirty_input.lower()
        clean_input = dirty_input.replace('\r', ' ').replace('\n', ' ')

        test.append(clean_input)

# number of data provided for each user is 50, nubmer of users is 2000
test = test[:100]

### URL, special characters and digits removal

In [147]:
for i in range(0, len(test)):
    test[i] = re.sub(r'http\S+', '', test[i])

    # remove special characters and digits
    test[i] = re.sub("(\\d|\\W)+"," ", test[i])

### Stemming

In [148]:
for i in range(0, len(test)):
    #tokenize
    splitted = word_tokenize(test[i])

    #stem
    for j in range(0, len(splitted)):
        splitted[j] = stemmer.stem(splitted[j])
    
    test[i] = splitted

### Stop-word removal

In [149]:
for i in range(0, len(test)):
    x = [w for w in test[i] if not w in stop_words]
    test[i] = " ".join(x)

### Predict the labels of the testing data

In [150]:
transformed_test = tdif_vec.transform(test)
predictions = clf.predict(transformed_test)
print(predictions)

['internationalNews' 'technology' 'society' 'internationalNews' 'politic'
 'internationalNews' 'technology' 'technology' 'localNews'
 'internationalNews' 'internationalNews' 'diverse' 'technology' 'sport'
 'diverse' 'sport' 'sport' 'culture' 'sport' 'politic' 'sport' 'politic'
 'technology' 'diverse' 'sport' 'economy' 'technology' 'culture'
 'technology' 'technology' 'internationalNews' 'diverse' 'technology'
 'technology' 'sport' 'diverse' 'culture' 'sport' 'economy'
 'internationalNews' 'culture' 'sport' 'culture' 'culture' 'technology'
 'economy' 'diverse' 'sport' 'diverse' 'technology' 'economy' 'culture'
 'technology' 'economy' 'politic' 'culture' 'economy' 'technology'
 'technology' 'localNews' 'culture' 'sport' 'economy' 'culture'
 'technology' 'culture' 'sport' 'diverse' 'politic' 'internationalNews'
 'culture' 'sport' 'technology' 'economy' 'technology' 'technology'
 'sport' 'politic' 'culture' 'culture' 'politic' 'culture' 'diverse'
 'economy' 'sport' 'technology' 'technology

### Calculate each user interests

In [151]:
articles_per_user = 50
interests = [{}]
user_index = 0
for i in range(0, len(predictions)):
    if predictions[i] not in interests[user_index].keys():
        interests[user_index][predictions[i]] = 0
    interests[user_index][predictions[i]] = interests[user_index][predictions[i]] + 1
    
    if i != 0 and i % articles_per_user == 0:
        user_index += 1
        interests.append({})
        
user_index = 0
for user in interests:
    user_interests = ('User {}:').format(user_index)
    for key, value in user.items():
        user_interests = ('{} {} {}%,').format(user_interests, key, (value*100.00/articles_per_user))
    user_index += 1
    print(user_interests)

User 0: internationalNews 14.0%, technology 24.0%, society 2.0%, politic 6.0%, localNews 2.0%, diverse 14.0%, sport 20.0%, culture 12.0%, economy 8.0%,
User 1: culture 24.0%, technology 28.0%, economy 10.0%, politic 8.0%, localNews 2.0%, sport 14.0%, diverse 8.0%, internationalNews 4.0%,
